<a href="https://colab.research.google.com/github/sridhar355/project1/blob/main/recommend_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix 
from sklearn.neighbors import NearestNeighbors
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter ("ignore")
from google.colab import files

import io
import nltk
nltk.download('stopwords')
#df2 = pd.read_csv(io.BytesIO(uploaded['Filename.csv']))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
uploaded = files.upload()

Saving details.csv to details.csv


In [ ]:

ratings=pd.read_csv(io.BytesIO(uploaded['details.csv']),names=['productid','productname','asins','brand','category','keys','manufacture','review-dates','reviews-dateadded','dateseen','datepurchased','dorecommend','review-id','review_num','ratings','url','text','reviews','usercity','userprov','userid'],index_col=0)
ratings=ratings.drop(['asins','brand','category','keys','manufacture','review-dates','reviews-dateadded','dateseen','datepurchased','dorecommend','review-id','url','text','usercity','userprov'],axis=1)
ratings=ratings.drop(['id'])
convert_dict = {'ratings': float ,
                'reviews': str ,
               }
ratings = ratings.astype(convert_dict)
ratings=ratings.fillna(0)
ratproducts=pd.read_csv("details.csv",names=['productid','productname','asins','brand','category','keys','manufacture','review-dates','reviews-dateadded','dateseen','datepurchased','dorecommend','review-id','review_num','ratings','url','text','reviews','usercity','userprov','userid'],index_col=1)
ratproducts=ratproducts.drop(['asins','brand','category','keys','manufacture','review-dates','reviews-dateadded','dateseen','datepurchased','review-id','url','text','usercity','userprov'],axis=1)
ratproduct=ratings['productname']
ratproduct=list(ratproduct)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,10,12,13,14,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
ratings_user=ratings.pivot_table(index=['productid'],columns=['userid'],values='ratings',fill_value=0)
rating_user=csr_matrix(ratings_user.values)
rating_user

<42x26790 sparse matrix of type '<class 'numpy.float64'>'
	with 30616 stored elements in Compressed Sparse Row format>

In [ ]:
knn_ratings=NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10)

knn_ratings.fit(rating_user)

cv=CountVectorizer(max_features=1500)

classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)

In [ ]:

def recommend(item,ratings,ds,model,n_recommendations):
    product_names = list(ds.index)
    product_ID_item = product_names.index(item)
    distance, indices=model.kneighbors(ds.iloc[product_ID_item,:].values.reshape(1,-1),n_neighbors=n_recommendations)
    list_ratings=[]
    for i in range(1,n_recommendations):
        list_ratings.append(ds.index[indices.flatten()[i]])
    for i in list_ratings:
        k1=ratings.loc[i].productname
        if type(k1)!=int and 0!=k1[0]:
            Recommend1.append(i)
    Recommend2=[]
    for i in Recommend1:
        if ratings.loc[i].productname[0] not in Recommend2:
            Recommend2.append(ratings.loc[i].productname[0])
    return Recommend2

In [ ]:
i="Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16 GB, Green Kid-Proof Case"

In [ ]:
Recommend2=[]
Recommend1=[]
k=ratproduct.count(i)
if(k>0):
    pid=ratproducts.loc[i].productid[0]
    Recommend2=recommend(pid,ratings,ratings_user,knn_ratings,10+1)

In [ ]:
#Recommendations based on ratings
Recommend2

['Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes Special Offers, Magenta',
 'All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi, 16 GB - Includes Special Offers, Magenta',
 'Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16 GB, Green Kid-Proof Case',
 'Echo (White),,,\r\nEcho (White),,,',
 'Amazon Kindle Paperwhite - eBook reader - 4 GB - 6 monochrome Paperwhite - touchscreen - Wi-Fi - black,,,',
 'Kindle Voyage E-reader, 6 High-Resolution Display (300 ppi) with Adaptive Built-in Light, PagePress Sensors, Wi-Fi - Includes Special Offers,',
 'Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes Special Offers, Black']

In [ ]:
def recommendation(Recommend,ratings):
    Recommend1={}
    for i in range(len(Recommend)):
        corpus=[]
        k=ratings.loc[Recommend[i]].reviews
        for j in range(len(k)):
            review = re.sub('[^a-zA-Z]', ' ', k[j] )
            review=review.lower()
            review=review.split() 
            ps=PorterStemmer()
            review=[ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
            review=' '.join(review)
            corpus.append(review)
        X=cv.fit_transform(corpus).toarray()
        y=list(ratings.loc[Recommend[i]].ratings)
        for j in range(len(y)):
            if y[j]>4:
                y[j]=1
            else:
                y[j]=0
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test)
        y_pred=list(y_pred)
        l=len(y_pred)
        c=y_pred.count(1)
        if((c/l)>=0.8):
            Recommend1[Recommend[i]]=float(c/l)
    Recommend2=[]
    Recommend=sorted(Recommend1.keys())
    for i in Recommend:
        if ratings.loc[i].productname[0] not in Recommend2:
            Recommend2.append(ratings.loc[i].productname[0])
    return Recommend2

In [ ]:
Recommend2=recommendation(Recommend1,ratings)

In [ ]:
#Recommendations after considering reviews 
Recommend2

['Amazon Kindle Paperwhite - eBook reader - 4 GB - 6 monochrome Paperwhite - touchscreen - Wi-Fi - black,,,',
 'Echo (White),,,\r\nEcho (White),,,',
 'Kindle Voyage E-reader, 6 High-Resolution Display (300 ppi) with Adaptive Built-in Light, PagePress Sensors, Wi-Fi - Includes Special Offers,',
 'Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes Special Offers, Black']